Data fra Ellen

In [142]:
import dhlab as dh
import dhlab.api.dhlab_api as api
import re
from scipy.spatial.distance import cosine as cos
import pandas as pd
import networkx as nx

import dhlab.graph_networkx_louvain as gnl

from collections import Counter

from IPython.display import Markdown

In [2]:
books = """Bjørnstjerne Bjørnson

· Synnøve Solbakken https://urn.nb.no/URN:NBN:no-nb_digibok_2014071726003

· Arne https://urn.nb.no/URN:NBN:no-nb_digibok_2016051026002

· Smaastykker https://urn.nb.no/URN:NBN:no-nb_digibok_2008071410003

Camilla Collett

· Amtmandens Døttre https://urn.nb.no/URN:NBN:no-nb_digibok_2009072112001

· Fortællinger https://urn.nb.no/URN:NBN:no-nb_digibok_2009081403009

· I de lange Nætter https://urn.nb.no/URN:NBN:no-nb_digibok_2006111400028

Jonas Lie

· Den Fremsynte https://urn.nb.no/URN:NBN:no-nb_digibok_2012060106021

· Fortællinger og Skildringer fra Norge https://urn.nb.no/URN:NBN:no-nb_digibok_2016062148006

· Lodsen og hans Hustru https://urn.nb.no/URN:NBN:no-nb_digibok_2014061108130

Andreas Munch

· Den Eensomme https://urn.nb.no/URN:NBN:no-nb_digibok_2019020548006

· Billeder fra Nord og Syd https://urn.nb.no/URN:NBN:no-nb_digibok_2016052029001

· En Aften paa Giske https://urn.nb.no/URN:NBN:no-nb_digibok_2021042848547

Rolf Olsen (dramatikere – vet ikke om det er aktuelt med skuespill?)

· Den sidste Viking https://urn.nb.no/URN:NBN:no-nb_digibok_2020022148515

· Kontrolløren https://urn.nb.no/URN:NBN:no-nb_digibok_2016060926003

· Anna Kolbjørnsdatter https://urn.nb.no/URN:NBN:no-nb_digibok_2007072402002

Henrik Wergeland

· For Almuen https://urn.nb.no/URN:NBN:no-nb_digibok_2019100726007

· Hassel-Nødder, med og uden Kjerne https://urn.nb.no/URN:NBN:no-nb_digibok_2008040210001

· Kong Carl Johans Historie https://urn.nb.no/URN:NBN:no-nb_digibok_2014030528001

Hugo Schwarz (pseudonym for Hanna Winsnes) · Grevens Datter https://urn.nb.no/URN:NBN:no-nb_digibok_2009110212001

· Det første Skridt https://urn.nb.no/URN:NBN:no-nb_digibok_2009111110004 · 

Til Jul https://urn.nb.no/URN:NBN:no-nb_digibok_2022062148540

Nicolai Ramm Østgaard

· En Fjeldbygd https://urn.nb.no/URN:NBN:no-nb_digibok_2014061108126

· Fra Skov og Fjeld https://urn.nb.no/URN:NBN:no-nb_digibok_2008110503010

· Østerdølen hjemkommen fra Amerika https://urn.nb.no/URN:NBN:no-nb_digibok_2012020208016"""

In [3]:
urns = re.findall("URN:NBN:no-nb_digibok_[0-9]+", books)

In [4]:
corpus = dh.Corpus()

In [5]:
corpus.extend_from_identifiers(urns)

In [6]:
test = corpus.frame
test.dhlabid = test.dhlabid.astype(int)
test.year = test.year.astype(int)

In [8]:
test

,urn,dhlabid,title,authors,oaiid,sesamid,isbn10,city,timestamp,year,publisher,langs,subjects,ddc,genres,literaryform,doctype,ocr_creator,ocr_timestamp
0,URN:NBN:no-nb_digibok_2006111400028,100614245,I de lange Nætter,"Collett , Camilla",oai:nb.bibsys.no:999407148844702202,37246857b59a4cadb5601eb2983c092c,,Christiania,18630101.0,1863,Cappelen,nob,,,,Uklassifisert,digibok,dhlab,20221201.0
1,URN:NBN:no-nb_digibok_2007072402002,100614914,Anna Kolbjørnsdatter : Skuespil i 3 Akter : me...,"Olsen , Rolf",oai:nb.bibsys.no:999422446214702202,01829285d9940ccf9c1009b6824b28c2,,Christiania,18530101.0,1853,Chr. Tønsbergs Forlag,nob,,,fiction,Skjønnlitteratur,digibok,dhlab,20221201.0
2,URN:NBN:no-nb_digibok_2008040210001,100615524,"Hassel-Nødder, med og uden Kjerne, dog til Tid...","Wergeland , Henrik / Tønsberg , Chr . / Fabrit...",oai:nb.bibsys.no:990114900124702202,041c54291705a6a548dc4b7c1478ef2b,,,18450101.0,1845,Chr. Tønsbergs Forlag. Trykt hos W.C Fabritius,dan,Norsk litteratur / 1800-tallet,,biography / short story,Skjønnlitteratur,digibok,dhlab,20221201.0
3,URN:NBN:no-nb_digibok_2008071410003,100615854,Smaastykker,"Bjørnson , Bjørnstjerne",oai:nb.bibsys.no:999513211454702202,0522549989d799b53ae0c12b2cb7fb85,,Bergen,18600101.0,1860,Giertsen's Forlag,nob,,,fiction,Skjønnlitteratur,digibok,dhlab,20221201.0
4,URN:NBN:no-nb_digibok_2008110503010,100616236,Fra Skov og Fjeld,"Østgaard , N. R.",oai:nb.bibsys.no:999324387844702202,07238830399a3b1f6d4e4514974df415,,Christiania,18580101.0,1858,Tønsbergs Forl.,nob,,,,Uklassifisert,digibok,dhlab,20221201.0
5,URN:NBN:no-nb_digibok_2009072112001,100629443,Amtmandens Døttre : en Fortælling,"Collett , Camilla",oai:nb.bibsys.no:990220947334702202,685249c68d39b323c57fb8cbe85b60b5,,Kjøbenhavn,18600101.0,1860,Gyldendal,nob,,,novel,Skjønnlitteratur,digibok,dhlab,20221209.0
6,URN:NBN:no-nb_digibok_2009081403009,100618711,Fortællinger,"Collett , Camilla",oai:nb.bibsys.no:999504726544702202,40fef0e364fa66c9397637d2b234ade8,,Christiania,18610101.0,1861,Steensballe,nob,norsk / skjønnlitteratur,839.82,short story,Skjønnlitteratur,digibok,dhlab,20221201.0
7,URN:NBN:no-nb_digibok_2009110212001,100619009,Grevens datter,"Winsnes , Hanna / Risum , Ludvig",oai:nb.bibsys.no:999408151764702202,b92063ce202bd7dfe6e2678c079677e4,,,18410101.0,1841,Forlagt af Guldberg & Dzwonkowski. Trykt i Gul...,dan,Norsk litteratur / 1800-tallet,,novel,Skjønnlitteratur,digibok,dhlab,20221201.0
8,URN:NBN:no-nb_digibok_2009111110004,100629839,Det første Skridt,"Winsnes , Hanna / Fabritius , W. C. ( Wilhelm ...",oai:nb.bibsys.no:999505623784702202,4f2d8c543cc6bdecaccb3bc4ee2af181,,,18440101.0,1844,Guldberg & Dzwonkowski. Trykt hos W.C. Fabritius,dan,Norsk litteratur / 1800-tallet,,novel,Skjønnlitteratur,digibok,dhlab,20221209.0
9,URN:NBN:no-nb_digibok_2012020208016,100630748,Østerdølen hemkømmen fraa Amerika,"Østgaard , N. R.",oai:nb.bibsys.no:999814450124702202,012cf177d4490bdd621537105682f4ec,,Christiania,18530101.0,1853,Feilberg & Landmark,nob,Emigranthistorie / Norsk-amerikanere / forente...,,,Uklassifisert,digibok,dhlab,20221209.0


In [269]:
uc = api.get_urn_frequencies(dhlabid=list(test.dhlabid))

In [9]:
test[test.authors.str.contains("erge")]

,urn,dhlabid,title,authors,oaiid,sesamid,isbn10,city,timestamp,year,publisher,langs,subjects,ddc,genres,literaryform,doctype,ocr_creator,ocr_timestamp
2,URN:NBN:no-nb_digibok_2008040210001,100615524,"Hassel-Nødder, med og uden Kjerne, dog til Tid...","Wergeland , Henrik / Tønsberg , Chr . / Fabrit...",oai:nb.bibsys.no:990114900124702202,041c54291705a6a548dc4b7c1478ef2b,,,18450101.0,1845,Chr. Tønsbergs Forlag. Trykt hos W.C Fabritius,dan,Norsk litteratur / 1800-tallet,,biography / short story,Skjønnlitteratur,digibok,dhlab,20221201.0
11,URN:NBN:no-nb_digibok_2014030528001,100631906,Kong Carl Johans Historie. Hvad Tidsrummet fra...,"Wergeland , Henrik / Malling , P. T. ( Peter T...",oai:nb.bibsys.no:990302647314702202,479ad2805b31864ac130dc68b8bc6d5a,,,18460101.0,1846,P.T. Mallings Forlags-Boghandel,dan,Historie / Personalhistorie / Union,,biography,Faglitteratur,digibok,dhlab,20221209.0
20,URN:NBN:no-nb_digibok_2019100726007,100635959,For Almuen. Femte Hefte,"Wergeland , Henrik / Krohn , Johan",oai:nb.bibsys.no:990302969604702202,0d6d27294bb2427b4fd6c685a3a3060d,,,18310101.0,1831,Trykt paa Forfatterens Forlag hos J. Krohn,dan,,,poetry,Skjønnlitteratur,digibok,dhlab,20221209.0


## Features

In [11]:
features = [x.strip() for x  in """og eller men 
ved med uten av for før för etter i på paa over under gjennom til hos 
igjen igen allerede ikke alltid aldri aldrig opp op ned ut inn ind oppe ope nede ute inne inde
at å aa om som hvis derfor 
vil ville skal skulle må maa 
hva hvor hvilken hvilket . , ; ! - ? 
en ei et den denne dette disse""".split()]


In [259]:
features = [x.strip() for x  in """og eller men 
ved med av for i over under til hos 
 ikke ned ut  nede ute at om som hvis derfor 
vil ville skal skulle  
hva hvor hvilken hvilket . , ; ! - ? 
en ei et den denne dette disse""".split()]
', '.join([x for x in features if x in tcounts.index])

'og, eller, men, ved, med, av, for, i, over, under, til, hos, ikke, ned, ut, nede, ute, at, om, som, hvis, derfor, vil, ville, skal, skulle, hvor, hvilken, hvilket, ., ,, ;, !, -, ?, en, ei, et, den, denne, dette, disse'

In [166]:
tcounts = dh.Counts(test).frame

In [175]:
tcounts_df = (tcounts/tcounts).fillna(0)

In [235]:
featureset = [x for x in tcounts.sum(axis = 1).head(200).index if tcounts_df.sum(axis=1)[x] == 23]
print(featureset)

[',', '.', 'i', 'at', 'jeg', 'en', 'den', 'af', 'var', 'der', 'paa', 'til', 'som', '!', 'de', 'et', 'for', 'han', 'men', 'har', 'om', '–', 'vi', 'ved', '?', 'da', 'os', 'sin', 'kan', 'over', 'vil', 'vilde', 'Du', ':', 'I', 'Men', 'kom', 'alle', 'De', 'Han', 'ind', 'op', 'her', 'gamle', '»', ';', 'vor', 'maa', 'hele', '«', 'Der', 'sine', 'Alle', 'store', "'", 'vel', 'stille', 'mange', 'Alt', 'lange', 'andre', 'ja', 'gjorde', 'Ingen', 'to', 'kunne', 'komme', 'hos']


In [410]:
small_set = [',', '.', 'i', 'at', 'og',
             'en', 'den', 'af', 'var', 'der', 'paa', 'til', 'som', '!', 'de', 'et', 'for', 
             'men', 'har', 'om', '–', 
             'ved', '?', 'da', 
             'sin', 'kan', 'over', 'vil', 'vilde', 
             ':', 'alle', 'De', 
             'ind', 'op', 'her', ';', 'vor', 'maa', 'hele', 'Der', 'sine', "'", 'vel', 'stille', 
            'andre', 'ja', 'gjorde', 'to', 'kunne', 'hos']

# Cooccurences

In [12]:
def colls(word, before = 0, after = 10, features = None):
    c = dict()
    for _, row in test[['dhlabid', 'urn']].iterrows():
        #print(dhid)
        c[row.dhlabid] = dh.Collocations([row.urn], words=[word], before = before, after = after).frame
        c[row.dhlabid] = c[row.dhlabid].loc[[x for x in features if x in c[row.dhlabid].index]].sort_values(by='counts', ascending = False)
        c[row.dhlabid].columns = [row.dhlabid]
    return c

In [15]:
def colls_features(before = 0, after = 10, features = None):
    c = dict()
    for word in features:
        print(word)
        c[word] = colls(word, before=before, after=after, features=features)
    return c

In [26]:
def store_tensor(tensor, file):
    """A dictionary of pandas dataframes"""
    dfs = dict()
    for t in tensor:
        dfs[t] = pd.concat([tensor[t][x] for x in tensor[t]], axis = 1).fillna(0).to_json()
        
    with open(file, "w") as fp:
        json.dump(dfs, fp)
        

In [63]:
def read_tensor(file):
    with open(file) as fp:
        dfs = json.load(fp)
    for x in dfs:
        dfs[x] = pd.read_json(dfs[x], convert_dates=False, dtype=int, convert_axes=False)
    return dfs

In [414]:
%%time
c = colls_features(before=0, after = 25, features=small_set)

,
.
i
at
og
en
den
af
var
der
paa
til
som
!
de
et
for
men
har
om
–
ved
?
da
sin
kan
over
vil
vilde
:
alle
De
ind
op
her
;
vor
maa
hele
Der
sine
'
vel
stille
andre
ja
gjorde
to
kunne
hos
CPU times: user 49.2 s, sys: 992 ms, total: 50.1 s
Wall time: 16min 6s


In [319]:
import json

In [320]:
embed = dict()
for e in c:
    embed[e] = pd.concat([c[e][x] for x in c[e]], axis = 1).fillna(0)

In [415]:
store_tensor(c, "tensor_b0_a25_small_set_2.json")

In [416]:
dfs = read_tensor( "tensor_b0_a25_small_set_2.json")

In [ ]:
#dfs = read_tensor( "tensor_b15_a0_features.json")

In [417]:
words = list({w for t in dfs for w in dfs[t].index})

# Get relative frequencies of words

In [418]:
wc = dh.Counts(test.urn, words = words).frame

In [419]:
wc

urn,100614245,100614914,100615524,100615854,100616236,100618711,100619009,100623480,100625114,100625322,...,100630748,100630920,100631906,100632122,100632125,100633224,100633323,100635959,100636170,100637015
word,,,,,,,,,,,,,,,,,,,,,
!,518.0,333.0,364.0,352.0,53.0,375.0,349.0,209.0,218.0,21.0,...,4.0,29.0,139.0,118.0,512.0,200.0,379.0,262.0,887.0,183.0
',48.0,3.0,37.0,66.0,78.0,14.0,22.0,14.0,124.0,6.0,...,97.0,12.0,11.0,701.0,10.0,34.0,64.0,21.0,55.0,9.0
",",5163.0,1175.0,3198.0,4362.0,3613.0,4731.0,9068.0,3184.0,3942.0,2018.0,...,123.0,3145.0,1860.0,6168.0,5676.0,6441.0,1178.0,1331.0,2254.0,876.0
.,2369.0,1201.0,2290.0,2810.0,1303.0,2189.0,3231.0,1562.0,1755.0,925.0,...,85.0,1146.0,966.0,2652.0,2788.0,1737.0,1464.0,843.0,1855.0,701.0
:,111.0,15.0,138.0,171.0,90.0,130.0,131.0,211.0,253.0,38.0,...,3.0,31.0,60.0,181.0,166.0,157.0,35.0,77.0,48.0,14.0
;,70.0,37.0,362.0,647.0,251.0,238.0,804.0,599.0,741.0,190.0,...,15.0,159.0,96.0,290.0,508.0,361.0,41.0,171.0,58.0,8.0
?,184.0,170.0,183.0,354.0,96.0,174.0,454.0,209.0,208.0,8.0,...,3.0,5.0,42.0,128.0,207.0,146.0,221.0,74.0,384.0,88.0
De,86.0,137.0,100.0,58.0,20.0,112.0,328.0,40.0,41.0,30.0,...,6.0,23.0,47.0,35.0,105.0,68.0,139.0,6.0,38.0,13.0
Der,62.0,22.0,49.0,36.0,25.0,35.0,55.0,43.0,39.0,27.0,...,6.0,26.0,6.0,26.0,86.0,23.0,15.0,5.0,24.0,12.0


## Relativize `dfs`

In [420]:
dfs['i'].columns

Index(['100614245', '100614914', '100615524', '100615854', '100616236',
       '100629443', '100618711', '100619009', '100629839', '100630748',
       '100630920', '100631906', '100632122', '100632125', '100623480',
       '100625114', '100633224', '100633323', '100625322', '100627229',
       '100635959', '100636170', '100637015'],
      dtype='object')

In [421]:
wc.columns = [str(x) for x in wc.columns]

In [422]:
def locup(series, key):
    try:
        res = series.loc[key]
    except:
        res = 1
    return res

In [424]:
locup(wc[wc.columns[0]], 'i')

1098.0

In [425]:
uci = uc.set_index('urn')

In [426]:
[uci.loc[int(u)].values[0] for u in dfs_rel['i'] if int(u) in uci.index]

[63140,
 17953,
 52569,
 52973,
 47348,
 126764,
 56672,
 103233,
 47895,
 2307,
 37962,
 28049,
 73290,
 78827,
 41167,
 44059,
 81788,
 19219,
 26098,
 32463,
 17972,
 31619,
 10991]

make the relativized as a pmi weighted array

In [438]:
dfs_rel = {i:pd.DataFrame({col:dfs[i][col]*uci.loc[int(col)].values[0]/(wc[col]*locup(wc[col],i)) for col in dfs[i]}).fillna(0) for i in dfs }

In [439]:
dfs_rel['vor']

,100614245,100614914,100615524,100615854,100616236,100629443,100618711,100619009,100629839,100630748,...,100632125,100623480,100625114,100633224,100633323,100625322,100627229,100635959,100636170,100637015
!,31.343629,17.970971,57.768132,0.000000,0.000000,21.437099,29.729574,35.495587,38.986569,0.000000,...,76.979492,65.657097,0.000000,21.810133,33.806508,0.000000,0.000000,16.332243,14.678228,12.870023
',18.791667,0.000000,0.000000,200.655303,0.000000,73.486377,0.000000,93.848182,0.000000,35.675258,...,0.000000,0.000000,355.314516,0.000000,0.000000,0.000000,0.000000,0.000000,33.817112,0.000000
",",27.952741,28.011773,23.013321,21.252350,26.209798,25.660124,23.761351,25.273187,22.086097,18.756098,...,22.567631,21.548890,33.530441,23.702986,21.753254,25.865213,26.403416,18.003506,26.405554,18.820205
.,22.464331,29.896753,25.251485,23.564502,27.253262,24.554627,27.587116,24.921616,39.192711,40.711765,...,17.671046,17.570209,0.000000,27.623658,24.067509,12.091737,37.313793,29.440660,20.053274,30.238129
:,8.126126,0.000000,0.000000,0.000000,0.000000,26.625499,14.293064,15.760763,0.000000,0.000000,...,59.357681,0.000000,0.000000,6.945902,0.000000,0.000000,70.571739,100.029685,0.000000,0.000000
;,25.771429,0.000000,58.087293,40.937403,47.159363,13.761494,3.903568,23.111866,26.129296,0.000000,...,58.189222,68.726210,0.000000,33.228735,0.000000,19.622556,27.165690,35.033138,64.135903,0.000000
?,24.510870,35.201961,0.000000,0.000000,0.000000,27.835749,16.018089,27.286256,13.739243,0.000000,...,0.000000,0.000000,0.000000,7.469224,28.987934,0.000000,0.000000,0.000000,4.843597,26.763799
De,20.976744,21.840633,0.000000,0.000000,0.000000,16.550986,41.475410,25.178780,21.315087,0.000000,...,0.000000,0.000000,0.000000,16.036863,23.044365,0.000000,0.000000,0.000000,0.000000,60.390110
Der,58.193548,0.000000,107.283673,0.000000,0.000000,15.702217,53.088525,75.078545,99.161491,0.000000,...,0.000000,319.124031,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
af,22.000000,0.000000,24.641719,0.000000,39.130579,22.383386,27.681167,8.473297,18.492278,0.000000,...,11.578584,0.000000,0.000000,14.789873,33.717544,13.859798,29.579043,23.938728,49.819853,19.148084


# Counts

In [512]:
#sub = tcounts.iloc[:70]
sub = tcounts.loc[[x for x in features if x in tcounts.index]]

In [ ]:
def distances(sub):
    dist = []
    for i in sub.columns:
        for j in sub.columns:
            dist.append([int(i),int(j), cos(sub[i], sub[j])])
    return pd.DataFrame(dist)

# Process counts or feature set

dtm from features, sub from set

In [ ]:
def make_distance_matrix(matrix, corpus = test):
    """Process a matrix, columns are dhlabid to be connected to corpus"""
    
    # compute the distances - in a small space, not efficient - check function..
    adj = distances(matrix)
    
    # extract author dhlabid columns
    auth = corpus[['dhlabid', 'authors']].copy()
    auth.dhlabid = auth.dhlabid.astype(int)
    
    adj[0] = adj[0].astype(int)
    adj[1] = adj[1].astype(int)
    
    # combine distances with metadata
    m = adj.merge(auth, left_on=0, right_on = 'dhlabid').merge(auth, left_on=1, right_on='dhlabid')
    
    m['one'] = m[0].astype(str) + '-' + m['authors_x'].apply(lambda x: x.split('/')[0])
    m['two'] = m[1].astype(str) + '-' + m['authors_y'].apply(lambda x: x.split('/')[0])



    m[['one','two', 2]]
    
    # Create a matrix of distances between books, col and row are books "dhlabid-author"
    res = m.pivot('one','two', 2)

    return res

In [ ]:
def evaluate_adjacency(adj, top=2):
    #adj.sort_values(by=adj.columns[0]).iloc[:3].index
    simdict = dict()
    for c in adj.columns:
        simdict[c] = list(adj.sort_values(by = c).iloc[:top+1].index)
    evaluation = dict()
    for x in simdict:
        evaluation[simdict[x][0]] = [z.split('-')[1].strip() for z in simdict[x][1:top]]
    return evaluation

# Run evaluation

In [449]:
def final_evaluation(adjs, top=2):
    final_eval = dict()
    evals = {x:evaluate_adjacency(adjs[x], top) for x in adjs}
    max_length = max([len(evals[x]) for x in evals])
    print(max_length)
    for x in evals:
        res = evals[x]
        if len(res) >= max_length:

            for x in res:
                if x in final_eval:
                    final_eval[x].update(res[x]) 
                else:
                    final_eval[x] = Counter(res[x])
    return final_eval

In [515]:
adjs = make_distance_matrix(sub)

In [516]:
adjs

two,"100614245-Collett , Camilla","100614914-Olsen , Rolf","100615524-Wergeland , Henrik","100615854-Bjørnson , Bjørnstjerne","100616236-Østgaard , N. R.","100618711-Collett , Camilla","100619009-Winsnes , Hanna","100623480-Bjørnson , Bjørnstjerne","100625114-Bjørnson , Bjørnstjerne","100625322-Lie , Jonas",...,"100630748-Østgaard , N. R.","100630920-Lie , Jonas","100631906-Wergeland , Henrik","100632122-Østgaard , N. R.","100632125-Lie , Jonas","100633224-Munch , A. ( Andreas )","100633323-Olsen , Rolf","100635959-Wergeland , Henrik","100636170-Olsen , Rolf","100637015-Munch , A. ( Andreas )"
one,,,,,,,,,,,,,,,,,,,,,
"100614245-Collett , Camilla",0.000000,0.081270,0.043539,0.046363,0.013825,0.002415,0.023455,0.083972,0.047436,0.029267,...,0.099146,0.015798,0.032470,0.014508,0.038973,0.019400,0.112693,0.036592,0.084492,0.080752
"100614914-Olsen , Rolf",0.081270,0.000000,0.072179,0.060956,0.130517,0.081446,0.118675,0.135517,0.115874,0.122920,...,0.102339,0.134457,0.108251,0.102565,0.101885,0.150395,0.010610,0.053300,0.022159,0.030626
"100615524-Wergeland , Henrik",0.043539,0.072179,0.000000,0.051159,0.046918,0.047194,0.055667,0.076879,0.082276,0.072067,...,0.129644,0.057844,0.064937,0.059474,0.059151,0.079999,0.087454,0.069321,0.096468,0.101677
"100615854-Bjørnson , Bjørnstjerne",0.046363,0.060956,0.051159,0.000000,0.062185,0.042789,0.037835,0.025345,0.022837,0.043972,...,0.070008,0.058770,0.066221,0.040480,0.035667,0.076899,0.082765,0.034404,0.058929,0.053127
"100616236-Østgaard , N. R.",0.013825,0.130517,0.046918,0.062185,0.000000,0.014461,0.022905,0.076067,0.057051,0.021677,...,0.111105,0.006390,0.029478,0.013558,0.032647,0.010449,0.168869,0.060307,0.137550,0.130634
"100618711-Collett , Camilla",0.002415,0.081446,0.047194,0.042789,0.014461,0.000000,0.018689,0.077043,0.040814,0.026234,...,0.098695,0.017289,0.029865,0.009991,0.039055,0.018568,0.111816,0.031275,0.089279,0.082409
"100619009-Winsnes , Hanna",0.023455,0.118675,0.055667,0.037835,0.022905,0.018689,0.000000,0.042268,0.027377,0.035045,...,0.123344,0.024725,0.050622,0.018955,0.035450,0.023560,0.157262,0.058140,0.118494,0.104550
"100623480-Bjørnson , Bjørnstjerne",0.083972,0.135517,0.076879,0.025345,0.076067,0.077043,0.042268,0.000000,0.038841,0.057769,...,0.109150,0.077437,0.087780,0.059931,0.041464,0.087846,0.167361,0.076972,0.127489,0.123667
"100625114-Bjørnson , Bjørnstjerne",0.047436,0.115874,0.082276,0.022837,0.057051,0.040814,0.027377,0.038841,0.000000,0.044338,...,0.107867,0.048838,0.071066,0.040710,0.049809,0.059391,0.146555,0.052344,0.105118,0.096232


In [528]:
def auth(w):
    return w.split('-')[1]

In [531]:
hits = {}
for x in adjs:
    #author = auth(x)
    print(x, [auth(z) for z in adjs[x].sort_values().index[1:4]])

100614245-Collett , Camilla ['Collett , Camilla', 'Collett , Camilla', 'Østgaard , N. R.']
100614914-Olsen , Rolf ['Olsen , Rolf ', 'Olsen , Rolf ', 'Munch , A. ( Andreas )']
100615524-Wergeland , Henrik  ['Winsnes , Hanna ', 'Collett , Camilla', 'Collett , Camilla']
100615854-Bjørnson , Bjørnstjerne ['Bjørnson , Bjørnstjerne', 'Bjørnson , Bjørnstjerne', 'Collett , Camilla']
100616236-Østgaard , N. R. ['Lie , Jonas', 'Munch , A. ( Andreas ) ', 'Østgaard , N. R.']
100618711-Collett , Camilla ['Collett , Camilla', 'Collett , Camilla', 'Østgaard , N. R.']
100619009-Winsnes , Hanna  ['Winsnes , Hanna ', 'Collett , Camilla', 'Østgaard , N. R.']
100623480-Bjørnson , Bjørnstjerne ['Bjørnson , Bjørnstjerne', 'Winsnes , Hanna ', 'Bjørnson , Bjørnstjerne']
100625114-Bjørnson , Bjørnstjerne ['Bjørnson , Bjørnstjerne', 'Winsnes , Hanna ', 'Winsnes , Hanna ']
100625322-Lie , Jonas ['Lie , Jonas', 'Lie , Jonas', 'Østgaard , N. R.']
100627229-Munch , A. ( Andreas )  ['Munch , A. ( Andreas ) ', 'Østga

In [466]:
adjs_rel = {w:make_distance_matrix(dfs_rel[w]) for w in dfs_rel if w in "og i af aa paa vil vilde kunne . ,".split()}

/home/larsj/anaconda3/lib/python3.9/site-packages/scipy/spatial/distance.py:622: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [496]:
adjs_rel = {w:make_distance_matrix(dfs_rel[w]) for w in dfs_rel }

/home/larsj/anaconda3/lib/python3.9/site-packages/scipy/spatial/distance.py:622: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [521]:
fe = final_evaluation(adjs_rel, top= 2)

23


In [508]:
def stats(fe):
    res = [(x.split('-')[1],fe[x].most_common(1)[0][0]) for x in fe]
    print(res)
    acc = len([x for x in res if x[0] == x[1]])*100/len(res)
    print("Accuracy", acc)
    authors = {x[0] for x in res}
    prec_recall = {x: 
                   { 'recall': len([z for z in res if x == z[1] and x==z[0]])/(1+len([z for z in res if x == z[0]])),
                    'precision': len([z for z in res if x == z[1] and x==z[0]])/(1+len([z for z in res if x == z[1]]))
                    } 
                   for x in authors
                  }
    print(prec_recall)

In [509]:
stats(fe)

[('Collett , Camilla', 'Collett , Camilla'), ('Olsen , Rolf', 'Olsen , Rolf'), ('Wergeland , Henrik ', 'Collett , Camilla'), ('Bjørnson , Bjørnstjerne', 'Munch , A. ( Andreas )'), ('Østgaard , N. R.', 'Collett , Camilla'), ('Collett , Camilla', 'Winsnes , Hanna'), ('Winsnes , Hanna ', 'Collett , Camilla'), ('Bjørnson , Bjørnstjerne', 'Bjørnson , Bjørnstjerne'), ('Bjørnson , Bjørnstjerne', 'Bjørnson , Bjørnstjerne'), ('Lie , Jonas', 'Bjørnson , Bjørnstjerne'), ('Munch , A. ( Andreas ) ', 'Bjørnson , Bjørnstjerne'), ('Collett , Camilla', 'Winsnes , Hanna'), ('Winsnes , Hanna ', 'Collett , Camilla'), ('Østgaard , N. R.', 'Olsen , Rolf'), ('Lie , Jonas', 'Collett , Camilla'), ('Wergeland , Henrik ', 'Collett , Camilla'), ('Østgaard , N. R.', 'Collett , Camilla'), ('Lie , Jonas', 'Collett , Camilla'), ('Munch , A. ( Andreas ) ', 'Collett , Camilla'), ('Olsen , Rolf ', 'Olsen , Rolf'), ('Wergeland , Henrik ', 'Olsen , Rolf'), ('Olsen , Rolf ', 'Olsen , Rolf'), ('Munch , A. ( Andreas )', 'Ols

In [511]:
uc.merge(test, left_on='urn', right_on='dhlabid')

,urn_x,freq,urn_y,dhlabid,title,authors,oaiid,sesamid,isbn10,city,...,year,publisher,langs,subjects,ddc,genres,literaryform,doctype,ocr_creator,ocr_timestamp
0,100614245,63140,URN:NBN:no-nb_digibok_2006111400028,100614245,I de lange Nætter,"Collett , Camilla",oai:nb.bibsys.no:999407148844702202,37246857b59a4cadb5601eb2983c092c,,Christiania,...,1863,Cappelen,nob,,,,Uklassifisert,digibok,dhlab,20221201.0
1,100614914,17953,URN:NBN:no-nb_digibok_2007072402002,100614914,Anna Kolbjørnsdatter : Skuespil i 3 Akter : me...,"Olsen , Rolf",oai:nb.bibsys.no:999422446214702202,01829285d9940ccf9c1009b6824b28c2,,Christiania,...,1853,Chr. Tønsbergs Forlag,nob,,,fiction,Skjønnlitteratur,digibok,dhlab,20221201.0
2,100615524,52569,URN:NBN:no-nb_digibok_2008040210001,100615524,"Hassel-Nødder, med og uden Kjerne, dog til Tid...","Wergeland , Henrik / Tønsberg , Chr . / Fabrit...",oai:nb.bibsys.no:990114900124702202,041c54291705a6a548dc4b7c1478ef2b,,,...,1845,Chr. Tønsbergs Forlag. Trykt hos W.C Fabritius,dan,Norsk litteratur / 1800-tallet,,biography / short story,Skjønnlitteratur,digibok,dhlab,20221201.0
3,100615854,52973,URN:NBN:no-nb_digibok_2008071410003,100615854,Smaastykker,"Bjørnson , Bjørnstjerne",oai:nb.bibsys.no:999513211454702202,0522549989d799b53ae0c12b2cb7fb85,,Bergen,...,1860,Giertsen's Forlag,nob,,,fiction,Skjønnlitteratur,digibok,dhlab,20221201.0
4,100616236,47348,URN:NBN:no-nb_digibok_2008110503010,100616236,Fra Skov og Fjeld,"Østgaard , N. R.",oai:nb.bibsys.no:999324387844702202,07238830399a3b1f6d4e4514974df415,,Christiania,...,1858,Tønsbergs Forl.,nob,,,,Uklassifisert,digibok,dhlab,20221201.0
5,100618711,56672,URN:NBN:no-nb_digibok_2009081403009,100618711,Fortællinger,"Collett , Camilla",oai:nb.bibsys.no:999504726544702202,40fef0e364fa66c9397637d2b234ade8,,Christiania,...,1861,Steensballe,nob,norsk / skjønnlitteratur,839.82,short story,Skjønnlitteratur,digibok,dhlab,20221201.0
6,100619009,103233,URN:NBN:no-nb_digibok_2009110212001,100619009,Grevens datter,"Winsnes , Hanna / Risum , Ludvig",oai:nb.bibsys.no:999408151764702202,b92063ce202bd7dfe6e2678c079677e4,,,...,1841,Forlagt af Guldberg & Dzwonkowski. Trykt i Gul...,dan,Norsk litteratur / 1800-tallet,,novel,Skjønnlitteratur,digibok,dhlab,20221201.0
7,100623480,41167,URN:NBN:no-nb_digibok_2014071726003,100623480,Synnøve Solbakken,"Bjørnson , Bjørnstjerne",oai:nb.bibsys.no:999404090114702202,cbb3a1a4d71faa3adc8cb1a179cf84d1,,Christiania,...,1857,Johan Dahl,nob,,,,Uklassifisert,digibok,dhlab,20221201.0
8,100625114,44059,URN:NBN:no-nb_digibok_2016051026002,100625114,Arne,"Bjørnson , Bjørnstjerne",oai:nb.bibsys.no:999112068844702202,49692694064385ffe6111e3c7c088d2f,,Bergen,...,1858,H.J. Geelmuydens Enke,nob,,,,Uklassifisert,digibok,dhlab,20221201.0
9,100625322,26098,URN:NBN:no-nb_digibok_2016062148006,100625322,Fortællinger og Skildringer fra Norge,"Lie , Jonas",oai:nb.bibsys.no:998330816594702202,6f832bb6919dd9e524317ac94111fdbe,,Kjøbenhavn,...,1872,Gyldendal,mul / nob / dan,,,,Uklassifisert,digibok,dhlab,20221201.0


In [468]:
results = ""
for e in fe:
    results += f"### {e}\n"
    results += '\n'.join([f"{x}: **{y}**\n" for (x, y) in fe[e].most_common()])
Markdown(results)

### 100614245-Collett , Camilla
Collett , Camilla: **3**

Munch , A. ( Andreas ): **2**

Lie , Jonas: **2**

Wergeland , Henrik: **1**
### 100614914-Olsen , Rolf
Collett , Camilla: **2**

Olsen , Rolf: **2**

Munch , A. ( Andreas ): **2**

Østgaard , N. R.: **1**

Winsnes , Hanna: **1**
### 100615524-Wergeland , Henrik 
Collett , Camilla: **3**

Bjørnson , Bjørnstjerne: **2**

Olsen , Rolf: **1**

Wergeland , Henrik: **1**

Lie , Jonas: **1**
### 100615854-Bjørnson , Bjørnstjerne
Bjørnson , Bjørnstjerne: **2**

Collett , Camilla: **2**

Winsnes , Hanna: **2**

Olsen , Rolf: **1**

Lie , Jonas: **1**
### 100616236-Østgaard , N. R.
Bjørnson , Bjørnstjerne: **2**

Munch , A. ( Andreas ): **2**

Collett , Camilla: **2**

Wergeland , Henrik: **1**

Olsen , Rolf: **1**
### 100618711-Collett , Camilla
Winsnes , Hanna: **2**

Wergeland , Henrik: **2**

Munch , A. ( Andreas ): **2**

Collett , Camilla: **2**
### 100619009-Winsnes , Hanna 
Collett , Camilla: **5**

Lie , Jonas: **1**

Winsnes , Hanna: **1**

Wergeland , Henrik: **1**
### 100623480-Bjørnson , Bjørnstjerne
Winsnes , Hanna: **2**

Munch , A. ( Andreas ): **2**

Wergeland , Henrik: **1**

Collett , Camilla: **1**

Bjørnson , Bjørnstjerne: **1**

Østgaard , N. R.: **1**
### 100625114-Bjørnson , Bjørnstjerne
Munch , A. ( Andreas ): **2**

Olsen , Rolf: **2**

Wergeland , Henrik: **1**

Lie , Jonas: **1**

Bjørnson , Bjørnstjerne: **1**

Collett , Camilla: **1**
### 100625322-Lie , Jonas
Lie , Jonas: **2**

Wergeland , Henrik: **2**

Bjørnson , Bjørnstjerne: **1**

Østgaard , N. R.: **1**

Winsnes , Hanna: **1**

Collett , Camilla: **1**
### 100627229-Munch , A. ( Andreas ) 
Bjørnson , Bjørnstjerne: **3**

Collett , Camilla: **2**

Olsen , Rolf: **1**

Winsnes , Hanna: **1**

Østgaard , N. R.: **1**
### 100629443-Collett , Camilla
Wergeland , Henrik: **3**

Winsnes , Hanna: **3**

Collett , Camilla: **1**

Munch , A. ( Andreas ): **1**
### 100629839-Winsnes , Hanna 
Winsnes , Hanna: **2**

Bjørnson , Bjørnstjerne: **2**

Lie , Jonas: **1**

Collett , Camilla: **1**

Wergeland , Henrik: **1**

Olsen , Rolf: **1**
### 100630748-Østgaard , N. R.
Bjørnson , Bjørnstjerne: **3**

Collett , Camilla: **2**

Munch , A. ( Andreas ): **1**

Wergeland , Henrik: **1**

Olsen , Rolf: **1**
### 100630920-Lie , Jonas
Collett , Camilla: **2**

Winsnes , Hanna: **2**

Wergeland , Henrik: **1**

Munch , A. ( Andreas ): **1**

Lie , Jonas: **1**

Bjørnson , Bjørnstjerne: **1**
### 100631906-Wergeland , Henrik 
Wergeland , Henrik: **3**

Collett , Camilla: **2**

Bjørnson , Bjørnstjerne: **1**

Lie , Jonas: **1**

Østgaard , N. R.: **1**
### 100632122-Østgaard , N. R.
Collett , Camilla: **3**

Wergeland , Henrik: **2**

Munch , A. ( Andreas ): **1**

Winsnes , Hanna: **1**

Bjørnson , Bjørnstjerne: **1**
### 100632125-Lie , Jonas
Collett , Camilla: **3**

Bjørnson , Bjørnstjerne: **2**

Winsnes , Hanna: **1**

Olsen , Rolf: **1**

Wergeland , Henrik: **1**
### 100633224-Munch , A. ( Andreas ) 
Collett , Camilla: **5**

Lie , Jonas: **2**

Wergeland , Henrik: **1**
### 100633323-Olsen , Rolf 
Olsen , Rolf: **3**

Wergeland , Henrik: **2**

Bjørnson , Bjørnstjerne: **1**

Collett , Camilla: **1**

Østgaard , N. R.: **1**
### 100635959-Wergeland , Henrik 
Olsen , Rolf: **3**

Bjørnson , Bjørnstjerne: **2**

Collett , Camilla: **1**

Østgaard , N. R.: **1**

Wergeland , Henrik: **1**
### 100636170-Olsen , Rolf 
Olsen , Rolf: **3**

Wergeland , Henrik: **2**

Collett , Camilla: **2**

Lie , Jonas: **1**
### 100637015-Munch , A. ( Andreas )
Bjørnson , Bjørnstjerne: **4**

Olsen , Rolf: **2**

Wergeland , Henrik: **1**

Collett , Camilla: **1**


In [ ]:
test.sort_values(by=["genres", "literaryform"], ascending=False)["dhlabid urn title authors year genres literaryform".split()]

In [230]:
from collections import Counter